# Perimetro Clientes Merge Analysis

This notebook contains the script to merge perimetro_clientes and query_results dataframes to create perimetro_completo with the columns aliasJ, SGC, and Status_GBO.

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, datetime

# Display all columns and more rows for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Sample Data Creation

Since the original dataframes are not available in this repository, I'll create sample data structures to demonstrate the merge functionality.

In [ ]:
# Create sample perimetro_clientes dataframe
# This represents the client perimeter data with GLCS field for matching
perimetro_clientes = pd.DataFrame({
    'GLCS': ['GLCS001', 'GLCS002', 'GLCS003', 'GLCS004', 'GLCS005', 'GLCS006'],
    'aliasJ': ['Cliente_A', 'Cliente_B', 'Cliente_C', 'Cliente_D', 'Cliente_E', 'Cliente_F'],
    'SGC': ['SGC_Alpha', 'SGC_Beta', 'SGC_Gamma', 'SGC_Delta', 'SGC_Epsilon', 'SGC_Zeta'],
    'Status_GBO': ['Active', 'Inactive', 'Pending', 'Active', 'Suspended', 'Active'],
    'additional_field1': ['Info1', 'Info2', 'Info3', 'Info4', 'Info5', 'Info6'],
    'additional_field2': [100, 200, 300, 400, 500, 600]
})

print("Sample perimetro_clientes dataframe:")
print(perimetro_clientes)
print(f"\nShape: {perimetro_clientes.shape}")

In [ ]:
# Create sample query_results dataframe (representing resultados_filtrados)
# This represents filtered query results with entityGLCS field for matching
resultados_filtrados = pd.DataFrame({
    'entityGLCS': ['GLCS001', 'GLCS002', 'GLCS003', 'GLCS007', 'GLCS008'],  # Note: some don't match
    'transaction_id': ['TXN001', 'TXN002', 'TXN003', 'TXN004', 'TXN005'],
    'amount': [1000.50, 2500.75, 1750.25, 3200.00, 950.80],
    'transaction_date': ['2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19'],
    'status': ['Completed', 'Pending', 'Completed', 'Failed', 'Completed']
})

print("Sample resultados_filtrados (query_results) dataframe:")
print(resultados_filtrados)
print(f"\nShape: {resultados_filtrados.shape}")

## Merge Operation

Now we'll perform the merge operation as specified:
1. Use pandas merge operation to perform a VLOOKUP-like join
2. Select only the required columns (aliasJ, SGC, Status_GBO)
3. Use the previously filtered resultados_filtrados dataframe
4. Join with perimetro_clientes based on the GLCS field

The merge will match `query_results['entityGLCS']` with `perimetro_clientes['GLCS']`.

In [ ]:
# Perform the merge operation
# Left join to keep all records from resultados_filtrados
# matching entityGLCS with GLCS
perimetro_completo = resultados_filtrados.merge(
    perimetro_clientes[['GLCS', 'aliasJ', 'SGC', 'Status_GBO']],  # Select only required columns
    left_on='entityGLCS',
    right_on='GLCS',
    how='left'  # Left join to keep all records from resultados_filtrados
)

print("Merged dataframe (perimetro_completo):")
print(perimetro_completo)
print(f"\nShape: {perimetro_completo.shape}")

## Analysis of Merge Results

In [ ]:
# Analyze the merge results
print("=== MERGE ANALYSIS ===")
print(f"Original resultados_filtrados records: {len(resultados_filtrados)}")
print(f"Original perimetro_clientes records: {len(perimetro_clientes)}")
print(f"Final perimetro_completo records: {len(perimetro_completo)}")

# Check for successful matches
successful_matches = perimetro_completo['aliasJ'].notna().sum()
print(f"\nSuccessful matches: {successful_matches}")
print(f"Unmatched records: {len(perimetro_completo) - successful_matches}")

# Show unmatched records
unmatched = perimetro_completo[perimetro_completo['aliasJ'].isna()]
if not unmatched.empty:
    print("\nUnmatched records (no corresponding GLCS in perimetro_clientes):")
    print(unmatched[['entityGLCS', 'transaction_id']])

# Show the final columns
print(f"\nFinal columns in perimetro_completo: {list(perimetro_completo.columns)}")

## Clean Final Dataset

Extract only the required columns as specified in the problem statement.

In [ ]:
# Create the final clean dataset with only the required columns from perimetro_clientes
# Plus keep the original data from resultados_filtrados for context
perimetro_completo_final = perimetro_completo[[
    'entityGLCS', 'transaction_id', 'amount', 'transaction_date', 'status',  # Original columns
    'aliasJ', 'SGC', 'Status_GBO'  # Required merged columns
]].copy()

print("Final perimetro_completo dataset:")
print(perimetro_completo_final)
print(f"\nShape: {perimetro_completo_final.shape}")
print(f"Columns: {list(perimetro_completo_final.columns)}")

## Summary and Export

The merge operation successfully combines the two dataframes using a VLOOKUP-like pandas merge.

In [ ]:
# Summary statistics
print("=== FINAL SUMMARY ===")
print(f"Total records in final dataset: {len(perimetro_completo_final)}")
print(f"Records with complete client data: {perimetro_completo_final['aliasJ'].notna().sum()}")
print(f"Records missing client data: {perimetro_completo_final['aliasJ'].isna().sum()}")

# Show data types
print("\nData types:")
print(perimetro_completo_final.dtypes)

# Optional: Save to CSV
# perimetro_completo_final.to_csv('perimetro_completo.csv', index=False)
# print("\nDataset saved to 'perimetro_completo.csv'")

## Reusable Function

Here's a reusable function to perform this merge operation with any similar datasets:

In [ ]:
def merge_perimetro_data(query_results_df, perimetro_clientes_df, 
                        query_glcs_col='entityGLCS', perimetro_glcs_col='GLCS',
                        required_cols=['aliasJ', 'SGC', 'Status_GBO']):
    """
    Merge query results with perimetro clientes data.
    
    Parameters:
    -----------
    query_results_df : pd.DataFrame
        The filtered query results dataframe
    perimetro_clientes_df : pd.DataFrame
        The client perimeter dataframe
    query_glcs_col : str
        Column name in query_results_df for GLCS matching (default: 'entityGLCS')
    perimetro_glcs_col : str
        Column name in perimetro_clientes_df for GLCS matching (default: 'GLCS')
    required_cols : list
        List of columns to extract from perimetro_clientes_df
    
    Returns:
    --------
    pd.DataFrame
        Merged dataframe with all original columns from query_results_df 
        plus required columns from perimetro_clientes_df
    """
    
    # Validate inputs
    if query_glcs_col not in query_results_df.columns:
        raise ValueError(f"Column '{query_glcs_col}' not found in query_results_df")
    
    if perimetro_glcs_col not in perimetro_clientes_df.columns:
        raise ValueError(f"Column '{perimetro_glcs_col}' not found in perimetro_clientes_df")
    
    missing_cols = [col for col in required_cols if col not in perimetro_clientes_df.columns]
    if missing_cols:
        raise ValueError(f"Required columns {missing_cols} not found in perimetro_clientes_df")
    
    # Prepare columns to merge
    merge_cols = [perimetro_glcs_col] + required_cols
    
    # Perform the merge
    merged_df = query_results_df.merge(
        perimetro_clientes_df[merge_cols],
        left_on=query_glcs_col,
        right_on=perimetro_glcs_col,
        how='left'
    )
    
    # Print merge statistics
    total_records = len(merged_df)
    successful_matches = merged_df[required_cols[0]].notna().sum()
    print(f"Merge completed: {successful_matches}/{total_records} records matched")
    
    return merged_df

# Test the function with our sample data
print("Testing reusable function:")
test_result = merge_perimetro_data(resultados_filtrados, perimetro_clientes)
print("\nFunction result:")
print(test_result[['entityGLCS', 'aliasJ', 'SGC', 'Status_GBO']].head())